# Modelamiento no supervisado en base a tópicos

In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import StemmerTokenizer

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

In [2]:
from utils.cargar import df_caso
from utils.preprocesamiento import process_df, procesar_adela

caso = 'adela'
df = df_caso(caso)

df = procesar_adela(df)
df = df[df['opt_left'] == 'Producir el alimento contra déficit vitamínico']

df = process_df(df,'comment','sel',verbose=True)

df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

26 rows found with non string elements for column comment (0.40%)
Deleting 685 columns for which max target value is over 7 (10.58%)
5761 available rows after processing


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

try:
    train_indices = pd.read_csv('../utils/splits/train_indices_{}'.format(caso), header=None)[0].tolist()
    test_indices = pd.read_csv('../utils/splits/test_indices_{}'.format(caso), header=None)[0].tolist()
    print("Partición train-test cargada")
except FileNotFoundError:
    train_indices, test_indices = train_test_split(range(len(df)), test_size=.2, stratify=df['sel'], random_state=0)
    pd.Series(train_indices).to_csv('../utils/splits/train_indices_{}'.format(caso), index=False)
    pd.Series(test_indices).to_csv('../utils/splits/test_indices_{}'.format(caso), index=False)
    print("Partición train-test guardada")

df_train = df.iloc[train_indices]
df_test = df.iloc[test_indices]

Partición train-test cargada


In [4]:
tokenized_corpus = [tokenizador(document) for document in df_train['comment']]
tokenized_test = [tokenizador(document) for document in df_test['comment']]

In [5]:
import gensim
from gensim import corpora

# Create a dictionary from the tokenized corpus
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the tokenized corpus into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

Usamos el número de tópicos con mejor métrica de coherencia

In [6]:
num_topics = 4

In [7]:
%%time
lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=num_topics, id2word=dictionary, passes=10)
print("Ajustando modelo")

Ajustando modelo
CPU times: user 13 s, sys: 0 ns, total: 13 s
Wall time: 13 s


In [8]:
# Print the generated topics
topics = lda_model.print_topics(num_topics=num_topics)
for topic in topics:
    print(topic)

(0, '0.030*"alimento" + 0.027*"personas" + 0.025*"tradiciones" + 0.019*"salud" + 0.015*"importante" + 0.012*"producir" + 0.011*"bien" + 0.010*"pueblo" + 0.009*"puede" + 0.008*"mas"')
(1, '0.034*"tradiciones" + 0.018*"alimento" + 0.018*"importante" + 0.013*"pueblos" + 0.012*"pueblo" + 0.011*"salud" + 0.010*"si" + 0.010*"debe" + 0.009*"producir" + 0.009*"identitarias"')
(2, '0.009*"ser" + 0.008*"tradiciones" + 0.008*"acuerdo" + 0.008*"mas" + 0.007*"si" + 0.007*"creencias" + 0.007*"puede" + 0.006*"grupo" + 0.006*"importante" + 0.005*"solo"')
(3, '0.037*"pueblo" + 0.020*"acuerdo" + 0.019*"llegar" + 0.019*"tradiciones" + 0.019*"originario" + 0.017*"fruta" + 0.016*"alimento" + 0.012*"si" + 0.011*"producción" + 0.010*"forma"')


In [9]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
pyLDAvis.display(vis_data)

In [10]:
print(' '.join(tokenized_test[0]) + '\n')

# Convert the tokenized document into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokenized_test[0])]

# Get the topic probabilities for the new document
topic_probs = lda_model.get_document_topics(doc_term_matrix)[0]

# Print the topic probabilities
for topic, prob in topic_probs:
    print(f"Topic {topic}: {prob}")

Adela debería priorizar salud personas alguna creencia grupo pequeño personas

Topic 0: 0.9286843538284302
Topic 1: 0.023898810148239136
Topic 2: 0.023312009871006012
Topic 3: 0.024104826152324677


## Correlaciones

In [11]:
from TM_utils import get_lda_embeddings
import pandas as pd

arr_test = get_lda_embeddings(lda_model,tokenizador,dictionary,df_test,'comment')
df_topics = pd.DataFrame(arr_test, columns=['Topic{}'.format(i+1) for i in range(len(topics))])

df_topics['sel'] = df_test['sel'].values

In [12]:
from scipy.stats import pearsonr


target_col_name = 'sel'
feature_target_corr = {}
for col in df_topics:
    if target_col_name != col:
        feature_target_corr[col + '_' + target_col_name] = \
            pearsonr(df_topics[col], df_topics[target_col_name])[0]

print("Feature-Target Correlations")
print(feature_target_corr)

Feature-Target Correlations
{'Topic1_sel': -0.33172544255561637, 'Topic2_sel': 0.095802492391719, 'Topic3_sel': -0.016489880848868613, 'Topic4_sel': 0.18328977118131276}


In [13]:
def get_correlations(lda_model,tokenizer,dictionary,df_test,num_topics,target_col_name='sel',verbose=True):
    arr_test = get_lda_embeddings(lda_model,tokenizer,dictionary,df_test,'comment')
    topics = lda_model.print_topics(num_topics=num_topics)
    df_topics = pd.DataFrame(arr_test, columns=['Topic{}'.format(i+1) for i in range(len(topics))])

    df_topics[target_col_name] = df_test[target_col_name].values

    feature_target_corr = {}
    for col in df_topics:
        if target_col_name != col:
            feature_target_corr[col + '_' + target_col_name] = \
                pearsonr(df_topics[col], df_topics[target_col_name])[0]

    if verbose:
        print("Feature-Target Correlations")
        print(feature_target_corr)
    
    return feature_target_corr

### Guardar Modelo

In [14]:
lda_model.save('modelos/LDA-{}_{}.gensim'.format(caso,num_topics))
print("Modelo guardado")

Modelo guardado


## Variando el número de tópicos

In [15]:
models = {}

for K in [10,25]:
    models[K] = gensim.models.LdaModel(doc_term_matrix, num_topics=K, id2word=dictionary, passes=10)
    get_correlations(models[K],tokenizador,dictionary,df_test,K)
    models[K].save('modelos/LDA-{}_{}.gensim'.format(caso,K))
    print("Modelo guardado con {} tópicos".format(K))

/home/camilo/miniconda3/envs/ethics_env/lib/python3.9/site-packages/gensim/models/ldamodel.py:850: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words


Feature-Target Correlations
{'Topic1_sel': 0.06488058972103243, 'Topic2_sel': 0.0912263687860681, 'Topic3_sel': 0.09622199722029705, 'Topic4_sel': 0.09202415027431013, 'Topic5_sel': 0.049267480293652424, 'Topic6_sel': 0.09885897071751243, 'Topic7_sel': 0.06552885281608599, 'Topic8_sel': 0.05942698085856356, 'Topic9_sel': 0.06363958596895743, 'Topic10_sel': -0.16947866393418862}
Modelo guardado con 10 tópicos


/home/camilo/miniconda3/envs/ethics_env/lib/python3.9/site-packages/gensim/models/ldamodel.py:850: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words


Feature-Target Correlations
{'Topic1_sel': 0.21323113178312678, 'Topic2_sel': 0.21323113178312678, 'Topic3_sel': 0.21323113178312678, 'Topic4_sel': 0.21323113178312678, 'Topic5_sel': 0.21323113178312678, 'Topic6_sel': 0.21323113178312678, 'Topic7_sel': 0.21323113178312678, 'Topic8_sel': 0.21323113178312678, 'Topic9_sel': 0.21323113178312678, 'Topic10_sel': 0.21323113178312678, 'Topic11_sel': 0.21323113178312678, 'Topic12_sel': -0.21323112966512717, 'Topic13_sel': 0.21323113178312678, 'Topic14_sel': 0.21323113178312678, 'Topic15_sel': 0.21323113178312678, 'Topic16_sel': 0.21323113178312678, 'Topic17_sel': 0.21323113178312678, 'Topic18_sel': 0.21323113178312678, 'Topic19_sel': 0.21323113178312678, 'Topic20_sel': 0.21323113178312678, 'Topic21_sel': 0.21323113178312678, 'Topic22_sel': 0.21323113178312678, 'Topic23_sel': 0.21323113178312678, 'Topic24_sel': 0.21323113178312678, 'Topic25_sel': 0.21323113178312678}
Modelo guardado con 25 tópicos
